# This did not end up working as I had hoped.
## Keeping here if need or interested

In [104]:
#pip install nba_api
#This did not end up working as I had hoped. Constantly got timeouts from the API and was not able to grab the data

In [102]:
#pip install pandas

In [91]:
# Always start with the imports 
import pandas as pd
import time
from nba_api.stats.static import players
from nba_api.stats.endpoints import PlayerCareerStats, LeagueDashPlayerStats
from nba_api.stats.endpoints import PlayerDashboardByYearOverYear

In [30]:
#Lets load in our contract data so we can get all of the unique player names.
df_contracts = pd.read_csv('Assets/NBA_Contracts.csv')
total_contracts = len(df_contracts)

# df_contracts.head() #Let's make sure things loaded in well 

In [18]:
#Now that we have our df_contracts we can create a list of unique player names
player_names = df_contracts['Player'].unique().tolist()
unique_players = len(player_names)

print(f'There are {unique_players} unique players of the {total_contracts} contracts, each will need a Player_ID')

There are 94 unique players of the 106 contracts, each will need a Player_ID


# NBA_API to get Player_ID
### Now that we have our list of unique player names from our contract data, we can use the players record from the api to get a dict of player IDs. We will create a function that takes in the list of unique player names and return a key-value pair of name-ID.

In [32]:
#Let's start by loading in our player data and creating it into a dict
all_players = players.get_players()
player_dict = {p['full_name']: p['id'] for p in all_players}

#Let's look at what information is contained in our play_dict
#We expect to get the players name and their unique ID for the api

# print(list(player_dict.items())[:3]) #Output: [('Alaa Abdelnaby', 76001), ('Zaid Abdul-Aziz', 76002), ('Kareem Abdul-Jabbar', 76003)] 
# print(f"Total players in dictionary: {len(player_dict)}") #Output: 4984 Players!

In [38]:
#Now that we have our player dict loaded in from the NBA API we can create our function
def get_player_IDs(player_names):

    
    matching_IDs = {} #Initalizing our dict that we will return
    unmatched_players = [] #Initalizing a list incase we run into unmatched players
    
    for name in player_names:
        player_id = player_dict.get(name, -999) #use the player name from the list to get the id value, if no id is found return -999 for the player id
        matching_IDs[name] = player_id

        if player_id == -999:
            unmatched_players.append(name)

    if unmatched_players:
        print(f'We have {len(unmatched_players)} players without IDs. Check the unmatched_players variable')
        print(unmatched_players)
            
    
    return matching_IDs

In [40]:
#Fingers crossed that we don't have any issues with player name matching!
player_IDs = get_player_IDs(player_names)

We have 4 players without IDs. Check the unmatched_players variable
['Nikola Vucevic', 'Jonas Valanciunas', 'C.J. McCollum', 'Louis Williams']


In [46]:
# Well we got most of our names but we ended up having 4 names that did not match with the player api database.
# These 4 names are fairly well know, and the reason we don't get an ID could be due to spelling or formatting errors 
    # For example: Vucevic above does not contain his unique character letters: Vučević
#Lets try to search for these players in the player_dict, and if we find them we can add them manually (since it's only 4)

for player in player_dict.keys():
    if "Vučević" in player or "Valančiūnas" in player or "McCollum" in player or "Lou Williams" in player:
        print(player)

#By running this cell we can see that the issue is due to the differences in player names and we can easily fix those now
#Louis Williams was specifically identified as his nickname above as their are many players with Williams as a last name
#and Louis Williams is generally known as Lou Williams

CJ McCollum
Jonas Valančiūnas
Nikola Vučević
Lou Williams


In [47]:
#Now that we know the correct spelling of these players names we can update our player_names and run our function again
fixed_names = {
    'Nikola Vucevic' : 'Nikola Vučević', 
    'Jonas Valanciunas' : 'Jonas Valančiūnas', 
    'C.J. McCollum' : 'CJ McCollum', 
    'Louis Williams' : 'Lou Williams'
}

player_names_fix = [fixed_names.get(name, name) for name in player_names] # use .get() to check if the name is in fixed_names,
                                                                          # if it is return the value (fixed name), if not return original (name we are looking up)

#Now we can rerun the function with our fixed names
#We could overwrite the other, of course, but we will keep it for reference
player_IDs_fix = get_player_IDs(player_names_fix)

In [88]:
#Lets take a peak at what we are outputting
print(list(player_IDs_fix.items())[:10])

[('Kevin Durant', 201142), ('Kobe Bryant', 977), ('Zach Randolph', 2216), ('Al Horford', 201143), ('Joakim Noah', 201149), ('Pau Gasol', 2200), ('Tony Parker', 2225), ('Mike Conley', 201144), ('Jared Dudley', 201162), ('Nick Collison', 2555)]


# NBA Player_ID to Player Stats


In [77]:
#Now that we have our player names and their respective IDs, we can use these IDs to look up their stats from the API

def get_player_stats(player_ID_dict):

    all_stats_player = []

    for name, player_ID in player_ID_dict.items():
        print(f'grabbing data for {name}')
        career_stats = PlayerCareerStats(player_id=player_ID, timeout=300) 
        df = career_stats.career_totals_regular_season.get_data_frames()

        #Lets make sure we include the player name for reference
        df["Player"] = name
        all_stats_player.append(df) #This is a list of dataframes (one for each players season stats)

        time.sleep(1) #adding a small 'nap' to our code to avoid overloading the api
        


    total_stats_players = pd.concat(all_stats_player, ignore_index=True) #Combining the list of dataframes into 1 combined one
    
    return total_stats_players

In [100]:
# The cell below is multiple test runs with different API features. The function above should run but seems to get stuck within the data retrieval

In [99]:
#player_stats_df = get_player_stats(player_IDs_fix)

#career_stats = PlayerDashboardByYearOverYear(player_id=player_IDs_fix["Cedi Osman"], season= '2020-2021', timeout=300)
#print(career_stats.get_data_frames()[0].head())